In [1]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd

In [2]:
path = r"C:\Users\Gowoon\Desktop\ASAC\ML프로젝트\web_crawling\LSH\steampy\2022_tag.csv"
data = pd.read_csv(path)
data = data.columns[1:].to_list()

In [16]:
normalized_text = []
for string in data:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [[sentence] for sentence in normalized_text]

In [17]:
result

[['gambling'],
 ['side scroller'],
 ['electronic music'],
 ['class based'],
 ['nature'],
 ['sailing'],
 ['driving'],
 ['alternate history'],
 ['lemmings'],
 ['wholesome'],
 ['2d platformer'],
 ['superhero'],
 ['e sports'],
 ['8 bit music'],
 ['golf'],
 ['voxel'],
 ['comic book'],
 ['rhythm'],
 ['dark humor'],
 ['early access'],
 ['silent protagonist'],
 ['rome'],
 ['gun customization'],
 ['combat'],
 ['feature film'],
 ['sequel'],
 ['vampire'],
 ['narrative'],
 ['cozy'],
 ['fast paced'],
 ['snow'],
 ['action rpg'],
 ['puzzle'],
 ['360 video'],
 ['emotional'],
 ['mature'],
 ['unforgiving'],
 ['coding'],
 ['turn based strategy'],
 ['wrestling'],
 ['grid based movement'],
 ['life sim'],
 ['time management'],
 ['parody '],
 ['metroidvania'],
 ['farming'],
 ['tactical'],
 ['skateboarding'],
 ['2 5d'],
 ['motocross'],
 ['volleyball'],
 ['boxing'],
 ['time attack'],
 ['flight'],
 ['fox'],
 ['medical sim'],
 ['destruction'],
 ['survival horror'],
 ['social deduction'],
 ['dating sim'],
 ['mod'

In [18]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 437


In [19]:
from gensim.models import Word2Vec
model = Word2Vec(result,min_count=1,workers=4, sg=1)
model.build_vocab(corpus_iterable=result)
model.train(corpus_iterable=result, total_examples=len(result), epochs=10)

(4370, 4370)

In [24]:
model.wv.most_similar("strategy")

[('nature', 0.2698708772659302),
 ('arcade', 0.2549944818019867),
 ('fast paced', 0.2508576214313507),
 ('nonlinear', 0.23967044055461884),
 ('competitive', 0.2393619418144226),
 ('pirates', 0.2146604210138321),
 ('action adventure', 0.214166522026062),
 ('diplomacy', 0.21201367676258087),
 ('rock music', 0.20948728919029236),
 ('detective', 0.2088119387626648)]

In [21]:
# 단어 유사도
model.wv.similarity('fishing', 'nature')

-0.0025548167

In [25]:
from sklearn.cluster import KMeans

word_vectors = model.wv.vectors # 어휘의 feature vector


In [26]:
word_vectors.shape

(437, 100)

In [30]:
num_clusters = int(word_vectors.shape[0]/30) # 어휘 크기의 1/5나 평균 5단어
print(num_clusters)
num_clusters = int(num_clusters)

14


In [31]:
kmeans_clustering = KMeans(n_clusters=num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

idx = list(idx)
names = model.wv.index_to_key
word_centroid_map = {names[i]: idx[i] for i in range(len(names))}

c:\anaconda\envs\edap\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [32]:
for c in range(num_clusters):
    # 클러스터 번호를 출력
    print("\ncluster {}".format(c))
    
    words = []
    cluster_values = list(word_centroid_map.values())
    for i in range(len(cluster_values)):
        if (cluster_values[i] == c):
            words.append(list(word_centroid_map.keys())[i])            
    print(words)


cluster 0
['mmorpg', 'shooter', 'game development', 'turn based combat', '3d fighter', 'snow', 'unforgiving', 'superhero', 'silent protagonist', 'rome', 'hand drawn', 'multiple endings', 'funny', 'character customization', 'shoot em up', 'snowboarding', 'video production', 'souls like', 'robots', 'rogue like', 'touch friendly', 'nostalgia', '2d', 'baseball', 'party based rpg', 'turn based tactics', 'memes', 'creature collector']

cluster 1
['historical', 'martial arts', 'psychological', 'tutorial', 'dystopian ', 'otome', 'philosophical', 'bikes', 'fast paced', 'boxing', '8 bit music', 'tanks', 'classic', 'dating sim', 'sandbox', 'assassin', 'villain protagonist', 'old school', 'agriculture', 'escape room', 'cartoon', 'animation modeling', 'america', 'warhammer 40k', 'time manipulation', 'tabletop', 'colony sim', 'ambient']

cluster 2
['fighting', 'futuristic', 'cricket', 'emotional', 'parody ', 'farming', 'family friendly', 'base building', 'party game', 'dark', 'massively multiplayer